# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fisterra,42.9049,-9.2629,10.97,70,51,13.30,ES,1709926313
1,1,yellowknife,62.4560,-114.3525,-9.21,66,40,0.00,CA,1709926313
2,2,lupane,-18.9315,27.8070,24.32,67,100,2.58,ZW,1709926313
3,3,bredasdorp,-34.5322,20.0403,19.07,75,0,1.21,ZA,1709926313
4,4,labuan,5.2767,115.2417,27.20,88,20,3.60,MY,1709926313


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [70]:
# Configure the map plot
map_of_cities = city_data_df.hvplot.points(
    "Lng",                  # Longitude points
    "Lat",                  # Latitude points
    geo = True,             # Plot on a geographic coordinate system
    tiles = "OSM",          # Use Open Street Maps
    color = "City",         # Different colours for each city
    alpha = 0.5,            # Sets Transparency at 50%
    size = "Humidity",      # Size of spot determinted by humidity
    scale = 0.5,            # Scaling of the spot
    frame_width = 800,      # Map image width
    frame_height = 500,     # Map image height
    xlabel = "Longitude",   # X Axis Label
    ylabel = "Latitude",    # Y Axis Label
    title = "Map of Select Cities with Humidity shown by size of spot"  # Title of Map
)

# Display the map
map_of_cities

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal weather conditions for holiday would be:
# 1. Temperature >25 and =<32 Degrees Celcius
# 2. Wind Speed < 4 m/s and
# 3. Cloudiness <= 30%

# Setting holiday conditions to variables
min_temp = 25
max_temp = 32
wind_speed = 4
cloudiness = 30

# Creating a filter for the ideal conditions
holiday_filter = (city_data_df['Max Temp'] > min_temp) & (city_data_df['Max Temp'] <= max_temp) & (city_data_df['Wind Speed'] < wind_speed) & (city_data_df['Cloudiness'] <=cloudiness)

# Creating the new dataframe
holiday_city_df = city_data_df[holiday_filter]


# Drop any rows with null values
holiday_city_df = holiday_city_df.dropna()

# Display sample data
holiday_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,labuan,5.2767,115.2417,27.20,88,20,3.60,MY,1709926313
6,6,inhambane,-23.8650,35.3833,27.99,83,20,3.09,MZ,1709926314
7,7,luanda,-8.8368,13.2343,28.00,83,0,2.57,AO,1709926216
31,31,adamstown,-25.0660,-130.1015,25.15,75,1,3.47,PN,1709925801
45,45,taoudenni,22.6783,-3.9836,27.76,9,11,2.32,ML,1709926318
57,57,belmonte,-15.8631,-38.8828,29.08,63,9,3.62,BR,1709925876
90,90,georgetown,5.4112,100.3354,28.86,88,20,1.03,MY,1709925872
118,118,san antonio de pale,-1.4014,5.6325,28.72,78,17,1.98,GQ,1709925868
173,173,el alto,-4.2714,-81.2147,30.81,66,24,3.87,PE,1709926336
185,185,fale old settlement,-9.3852,-171.2468,28.72,73,14,2.84,TK,1709925873


### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = holiday_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data & reseting the index
hotel_df.reset_index(drop=True)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,labuan,MY,5.2767,115.2417,88,
1,inhambane,MZ,-23.8650,35.3833,83,
2,luanda,AO,-8.8368,13.2343,83,
3,adamstown,PN,-25.0660,-130.1015,75,
4,taoudenni,ML,22.6783,-3.9836,9,
5,belmonte,BR,-15.8631,-38.8828,63,
6,georgetown,MY,5.4112,100.3354,88,
7,san antonio de pale,GQ,-1.4014,5.6325,78,
8,el alto,PE,-4.2714,-81.2147,66,
9,fale old settlement,TK,-9.3852,-171.2468,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key,    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
labuan - nearest hotel: Labuan Avenue Hotel
inhambane - nearest hotel: Hotel de Inhambane
luanda - nearest hotel: Hotel Forum
adamstown - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
san antonio de pale - nearest hotel: No hotel found
el alto - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
city of balikpapan - nearest hotel: Hotel Djang Djaya 2
lorengau - nearest hotel: Seeadler Bay Hotel
hadibu - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
san andres - nearest hotel: Hotel Decameron Isleño
cochin - nearest hotel: Hotel Top Form
mahina - nearest hotel: Motu Martin
saint-pierre - nearest hotel: Tropic Hotel
ad dabbah - nearest hotel: No hotel found
galle - nearest hotel: Jungle Hostel Villa
gudlavalleru - nearest hotel: No hotel found
colombo - nearest hotel: City Beds - The Regent
batan

,City,Country,Lat,Lng,Humidity,Hotel Name
4,labuan,MY,5.2767,115.2417,88,Labuan Avenue Hotel
6,inhambane,MZ,-23.8650,35.3833,83,Hotel de Inhambane
7,luanda,AO,-8.8368,13.2343,83,Hotel Forum
31,adamstown,PN,-25.0660,-130.1015,75,No hotel found
45,taoudenni,ML,22.6783,-3.9836,9,No hotel found
57,belmonte,BR,-15.8631,-38.8828,63,No hotel found
90,georgetown,MY,5.4112,100.3354,88,Page 63 hostel
118,san antonio de pale,GQ,-1.4014,5.6325,78,No hotel found
173,el alto,PE,-4.2714,-81.2147,66,No hotel found
185,fale old settlement,TK,-9.3852,-171.2468,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [71]:
# Removing cities where no hotels were found
cleaned_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != "No hotel found"]

# Configure the map plot
map_of_hotels = cleaned_hotel_df.hvplot.points(
    "Lng",                  # Longitude points
    "Lat",                  # Latitude points
    geo = True,             # Plot on a geographic coordinate system
    tiles = "OSM",          # Use Open Street Maps
    color = "City",         # Different colours for each city
    alpha = 0.5,            # Sets Transparency at 50%
    frame_width = 700,      # Map image width
    frame_height = 500,     # Map image height
    xlabel = "Longitude",   # X Axis Label
    ylabel = "Latitude",    # Y Axis Label
    hover_cols = ['Hotel Name', 'Country'],
    title = "Map of Hotels for perfect holiday"  # Title of Map
)
# Display the map
map_of_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)